In [2]:
import numpy as np
import os
from glob import glob
import csv
import pandas as pd

In [5]:
unmix = glob(os.path.join('eval_unmix', '**/raw_results.csv'), recursive=True)[0]
mix = glob(os.path.join('eval_mix', '**/raw_results.csv'), recursive=True)[0]

In [6]:
um_data = pd.read_csv(unmix)
m_data = pd.read_csv(mix)

In [7]:
um_data

,Unnamed: 0,0,1
0,0,Data/catridge/test/deform/039.png,4.294325
1,1,Data/catridge/test/deform/003.png,5.506176
2,2,Data/catridge/test/deform/044.png,4.572932
3,3,Data/catridge/test/deform/030.png,4.200860
4,4,Data/catridge/test/deform/081.png,5.174550
...,...,...,...
755,755,Data/catridge/test/debris/100.png,3.258304
756,756,Data/catridge/test/debris/115.png,4.666994
757,757,Data/catridge/test/debris/160.png,3.076677
758,758,Data/catridge/test/debris/032.png,2.796059


In [8]:
m_data

,Unnamed: 0,0,1
0,0,Data/catridge/test/deform/039.png,4.294325
1,1,Data/catridge/test/deform/003.png,5.506176
2,2,Data/catridge/test/deform/044.png,4.572932
3,3,Data/catridge/test/deform/030.png,4.200860
4,4,Data/catridge/test/deform/081.png,5.174550
...,...,...,...
775,775,Data/catridge/test/debris/100.png,3.258327
776,776,Data/catridge/test/debris/115.png,4.666982
777,777,Data/catridge/test/debris/160.png,3.076675
778,778,Data/catridge/test/debris/032.png,2.796076


In [20]:
def makeStat(data):
    stat = {}
    for img_dir, dist in zip(data['0'], data['1']):
        defect = img_dir.split('/')[-2]
        if defect in stat.keys():
            stat[defect].append([img_dir, dist])
        else:
            stat[defect] = [[img_dir, dist]]
            
    mean_std = {}
    for key in stat.keys():
        d = []
        for img_dir, dist in stat[key]:
            d.append(dist)
        d = np.array(d)
        mean = np.mean(d)
        std = np.std(d)
        mean_std[key] = [mean, std]
    
    stat['MS'] = mean_std
    
    return stat

In [21]:
um_stat = makeStat(um_data)
m_stat = makeStat(m_data)

In [22]:
um_stat['MS']

{'deform': [4.5081515646706585, 1.543217407330838],
 'crack': [3.2582912, 0.2481177407396839],
 'mist': [5.100712979545455, 2.3570753165497047],
 'angle': [7.438534135483872, 5.711074101676576],
 'broken': [3.63424053699422, 0.7376581672911529],
 'good': [3.1052310875, 0.9854301562985277],
 'debris': [4.172389109411765, 1.3039771382031125]}

In [23]:
m_stat['MS']

{'deform': [4.508151988023952, 1.5432172166157399],
 'crack': [3.2582912, 0.2481177407396839],
 'mist': [5.100713253409091, 2.35707515280761],
 'angle': [7.438533440322582, 5.711073627075399],
 'broken': [3.634240676878612, 0.7376581963020677],
 'good': [2.949780057142857, 0.8299422275421409],
 'debris': [4.172389504117647, 1.303976696445045]}

In [24]:
um_np = np.array(um_data)

In [30]:
um_bads = np.where(um_np[:,2] > um_stat['MS']['good'][0])[0]

In [32]:
len(um_bads)

610

In [34]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(um_np)):
    idx, path, dist = um_np[i]
    defect = path.split('/')[-2]
    
    if idx in um_bads and defect != 'good':
        tp += 1
    elif idx in um_bads and defect == 'good':
        fp += 1
    elif not idx in um_bads and defect != 'good':
        fn += 1
    else:
        tn += 1

In [35]:
tp

607

In [36]:
tn

5

In [37]:
fp

3

In [38]:
fn

145